In [1]:
import pandas as pd
import random 
import numpy as np
from scipy.spatial import distance_matrix
import time
import warnings
from tqdm import tqdm 
import itertools
warnings.filterwarnings('ignore')

In [2]:
tsp_c = pd.read_csv("TSPC.csv", sep=';', header=None)
tsp_d = pd.read_csv("TSPD.csv", sep=';', header=None)

In [3]:
def generate_dm(df, show=True):
    temp = df[[0, 1]].to_numpy()
    dm = distance_matrix(temp, temp)


    temp = df[2].to_numpy() // 2
    temp = temp * np.ones((200, 200))
    dm = dm+temp+temp.T
    dm = dm // 1

    for i in range(200):
        dm[i][i] = np.inf

    
    if show: 
        df_dist = pd.DataFrame(dm)
        display(df_dist)
    return dm


all_posibble_comb = list(itertools.combinations([i for i in range(100)], 2))
dm = generate_dm(tsp_c)


,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,inf,1690.0,1022.0,997.0,672.0,2250.0,2008.0,1903.0,2522.0,1721.0,...,2292.0,2845.0,1067.0,738.0,1391.0,1097.0,2232.0,3084.0,2821.0,1261.0
1,1690.0,inf,1420.0,2642.0,1103.0,1730.0,2081.0,3212.0,1476.0,1255.0,...,1986.0,1996.0,1002.0,2383.0,2161.0,2485.0,1572.0,2372.0,1404.0,664.0
2,1022.0,1420.0,inf,1976.0,436.0,2087.0,2057.0,2599.0,2263.0,1479.0,...,2207.0,2627.0,821.0,1705.0,1712.0,1833.0,2028.0,2909.0,2532.0,1056.0
3,997.0,2642.0,1976.0,inf,1627.0,3131.0,2772.0,2205.0,3450.0,2652.0,...,3128.0,3750.0,2020.0,1115.0,2059.0,1438.0,3135.0,3960.0,3776.0,2203.0
4,672.0,1103.0,436.0,1627.0,inf,1792.0,1762.0,2277.0,1965.0,1183.0,...,1913.0,2331.0,493.0,1358.0,1405.0,1506.0,1734.0,2614.0,2222.0,725.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1097.0,2485.0,1833.0,1438.0,1506.0,2434.0,1798.0,903.0,2900.0,2220.0,...,2295.0,3092.0,1924.0,1301.0,1069.0,inf,2513.0,3196.0,3487.0,2159.0
196,2232.0,1572.0,2028.0,3135.0,1734.0,768.0,1280.0,3052.0,985.0,1351.0,...,1044.0,1205.0,1821.0,2847.0,1905.0,2513.0,inf,1475.0,1875.0,1783.0
197,3084.0,2372.0,2909.0,3960.0,2614.0,1391.0,1570.0,3614.0,1154.0,2236.0,...,1287.0,1119.0,2706.0,3675.0,2544.0,3196.0,1475.0,inf,2291.0,2646.0
198,2821.0,1404.0,2532.0,3776.0,2222.0,2045.0,2565.0,4146.0,1275.0,2066.0,...,2306.0,1913.0,2144.0,3507.0,3011.0,3487.0,1875.0,2291.0,inf,1827.0


In [4]:
def get_random_solution():
    return random.sample([i for i in range(0, 200)], 100)
path = get_random_solution()
len(path)

100

In [5]:
def calulate_total_cost(path):
    total = 0
    nr = len(path)
    for idx, node in enumerate(path):
        total += dm[node][path[(idx+1)%nr]]
    return total
calulate_total_cost(path)

200163.0

In [6]:
def generate_inter_moves(path, max_id):
#     print(path)
    not_included_ids = list(set([i for i in range(200)]) - set(path))
    inter_moves = []
    for idx, node in enumerate(path):
        for new_node in not_included_ids:
            new_path = path[:idx] + [new_node] + path[idx+1:]
#             print((idx+1)%200)
            new_cost = dm[path[idx-1]][new_node] + dm[path[(idx+1)%100]][new_node] # two new edges
            new_cost -= (dm[path[idx-1]][node] + dm[path[(idx+1)%100]][node]) # two old edges
            # we want to min new_cost
            
            inter_moves.append((new_path, new_cost))
    return inter_moves
    
len(generate_inter_moves(path, 200))

10000

In [7]:
def generate_intra_moves(path, type_of_change):
    intra_moves = []
    if type_of_change == 'node':
        for idx1, idx2 in all_posibble_comb:
            new_path = path[:idx1]+[path[idx2]]+path[idx1+1:idx2]+[path[idx1]]+path[idx2+1:]
            new_cost = dm[path[idx1]][path[(idx2+1)%100]] + dm[path[idx1]][path[idx2-1]]   #new edges for node1
            new_cost += dm[path[idx2]][path[(idx1+1)%100]] + dm[path[idx2]][path[idx1-1]]  #new edges for node2
            new_cost -= (dm[path[idx1]][path[idx1-1]] + dm[path[idx1]][path[(idx1+1)%100]])#old edges for node1
            new_cost -= (dm[path[idx2]][path[idx2-1]] + dm[path[idx2]][path[(idx2+1)%100]])#old edges for node2
#             print(idx1, idx2, new_path)
            intra_moves.append((new_path, new_cost))
    else: #edge
        
        for idx1, idx2 in all_posibble_comb:
            if (idx2+1)%100 != idx1:
                new_path = path[:idx1] + path[idx1:idx2+1][::-1] + path[idx2+1:] # %100 is const.
#                 new_path = path[:idx1] + path[idx1:idx2][::-1] + path[idx2:] # %100 is const.
                new_cost = dm[path[idx1]][path[(idx2+1)%100]] + dm[path[idx1-1]][path[idx2]] #new edges
                new_cost -= (dm[path[idx1-1]][path[idx1]] + dm[path[idx2]][path[(idx2+1)%100]]) #old edges
                intra_moves.append((new_path, new_cost))
                         
    return intra_moves
# all_posibble_comb = list(itertools.combinations([i for i in range(100)], 2))
# len(generate_intra_moves(path, 'node'))

In [8]:
def main(type_of_change, greedy, path):    
    change = True
    max_id = dm.shape[0]

    while change:
        change = False
        # possible_moves [(new_path, delta)]
        # inter: change set of selected nodes(one in ine out)
        # intra: change order of nodes
            # a) exchange 2 nodes
            # b) exchange 2 edges
        possible_moves = generate_inter_moves(path, max_id) + generate_intra_moves(path, type_of_change)
        if greedy:
            random.shuffle(possible_moves)
            for new_path, new_cost in possible_moves:
                if new_cost < 0:
#                     print(calulate_total_cost(path))
#                     print(new_cost)
                    path = new_path
#                     print(calulate_total_cost(path))
                    change = True
                    break
        else:
            possible_moves.sort(key= lambda x: x[1])
            if possible_moves[0][1] < 0:
                path = possible_moves[0][0]
#                 print(calulate_total_cost(path))
                change = True
                
        
    return path

In [9]:
# path = get_random_solution()
# main('edge', True, path)

In [10]:
from tqdm import tqdm
from multiprocessing import Pool

In [ ]:
times = {}
solution = {}

def run(type_of_change, greedy, path):
    global times, solution
    start_time = time.time()
    final_path = main(type_of_change, greedy, path)
    final_cost = calulate_total_cost(final_path)
    times[type_of_change][str(greedy)].append(time.time()-start_time)
    solution[type_of_change][str(greedy)].append(final_cost)
    
for type_of_change in ['node', 'edge']:
    times[type_of_change] = {}
    solution[type_of_change] = {}
    for greedy in [True, False]:
        times[type_of_change][str(greedy)] = []
        solution[type_of_change][str(greedy)] = []
        print(type_of_change, greedy)
        pool = Pool(processes=4)
        for i in range(10):
            path = get_random_solution()
            pool.apply_async(run, args=(type_of_change, greedy, path,))
        pool.close()
        pool.join()

node True


-434.0
219845.0
-135.0
217727.0
-3432.0
214295.0
-811.0
213484.0
-597.0
212887.0
-1422.0
211465.0
-843.0
210622.0
-1533.0
209089.0
-343.0
211274.0
-884.0
210390.0
-659.0
209731.0
-5188.0
204543.0
-2695.0
204389.0
-2306.0
202083.0
-1076.0
201007.0
-1174.0
200640.0
-1529.0
199111.0
-58.0
198087.0
-2456.0
197576.0
-2062.0
195514.0
-241.0
195273.0
-2657.0
192616.0
-1288.0
193312.0
-91.0
191414.0
-1606.0
189808.0
-881.0
190523.0
-1580.0
188943.0
-3594.0
185349.0
-710.0
185365.0
-1911.0
183454.0
-125.0
183329.0
-2997.0
180332.0
-1445.0
180245.0
-1463.0
180309.0
-2453.0
177856.0
-667.0
179875.0
-1936.0
177939.0
-310.0
176280.0
-1473.0
178351.0
-521.0
177830.0
-1651.0
176179.0
-2239.0
176142.0
-579.0
175563.0
-1453.0
174110.0
-758.0
173352.0
-209.0
173143.0
-415.0
172192.0
-247.0
172618.0
-518.0
172100.0
-21.0
172079.0
-261.0
172216.0
-564.0
168939.0
-222.0
172114.0
-1093.0
171424.0
-1713.0
169711.0
-140.0
169571.0
-201.0
170355.0
-263.0
169059.0
-672.0
170474.0
-1676.0
168798.0
-97.0
168167.0

-60.0
80290.0
-144.0
80146.0
-471.0
79675.0
-268.0
79407.0
-350.0
79407.0
-308.0
79298.0
-502.0
78666.0
-17.0
79205.0
-315.0
78890.0
-23.0
78867.0
-725.0
78142.0
-169.0
77973.0
-732.0
77241.0
-78.0
77163.0
-471.0
77496.0
-830.0
76666.0
-136.0
76490.0
-222.0
76560.0
-125.0
76435.0
-551.0
75884.0
-162.0
75884.0
-303.0
76173.0
-100.0
76073.0
-46.0
76065.0
-297.0
76186.0
-8.0
76178.0
-10.0
76349.0
-27.0
76322.0
-3.0
76319.0
-43.0
76276.0
-261.0
76015.0
-429.0
75586.0
-124.0
75705.0
-119.0
75586.0
-24.0
75586.0
-295.0
75291.0
-271.0
75543.0
-718.0
75570.0
-310.0
75260.0
-156.0
75253.0
-289.0
75453.0
-44.0
75409.0
-657.0
75443.0
-439.0
75872.0
-163.0
75709.0
-67.0
75388.0
-54.0
75334.0
-131.0
75203.0
-210.0
74993.0
-76.0
75691.0
-216.0
75475.0
-31.0
75704.0
-113.0
75591.0
-70.0
75135.0
-487.0
75182.0
-20.0
75164.0
-356.0
74808.0
-460.0
75069.0
-141.0
74928.0
-159.0
75095.0
-13.0
75082.0
-11.0
75071.0
-335.0
74736.0
-269.0
74349.0
-237.0
74112.0
-215.0
73897.0
-87.0
73810.0
-281.0
74330.0
-36

-240.0
57159.0
-234.0
57159.0
-186.0
56973.0
-340.0
56986.0
-83.0
56986.0
-82.0
56986.0
-406.0
57143.0
-78.0
57108.0
-341.0
57159.0
-42.0
57159.0
-13.0
58045.0
-82.0
58045.0
-13.0
58032.0
-106.0
57926.0
-727.0
58092.0
-229.0
58092.0
-444.0
58050.0
-435.0
57615.0
-11.0
57604.0
-156.0
57448.0
-138.0
57210.0
-127.0
57083.0
-103.0
57085.0
-14.0
57047.0
-217.0
57105.0
-222.0
56883.0
-158.0
56751.0
-286.0
56465.0
-125.0
56340.0
-26.0
56380.0
-48.0
56415.0
-42.0
56373.0
-16.0
56357.0
-12.0
57196.0
-428.0
56768.0
-174.0
56768.0
-58.0
56768.0
-132.0
56636.0
-87.0
56392.0
-155.0
56237.0
-46.0
56492.0
-210.0
56282.0
-264.0
56018.0
-158.0
55969.0
-14.0
55701.0
-24.0
55701.0
-143.0
55773.0
-35.0
55773.0
-174.0
55599.0
-265.0
55599.0
-32.0
55599.0
-20.0
55599.0
-108.0
55599.0
-259.0
55945.0
-265.0
55945.0
-104.0
55841.0
-53.0
55667.0
-82.0
55655.0
-256.0
55939.0
-18.0
55921.0
-44.0
56029.0
-54.0
55975.0
-91.0
56143.0
-217.0
55886.0
-636.0
55883.0
-70.0
56264.0
-46.0
56521.0
-58.0
56463.0
-77.0
56386

-355.0
51652.0
-538.0
51710.0
-2.0
51710.0
-17.0
51572.0
-213.0
51359.0
-11.0
51438.0
-1.0
51438.0
-2.0
51438.0
-1.0
51438.0
-546.0
51083.0
-118.0
51083.0
-118.0
51083.0
-10.0
51083.0
-94.0
51083.0
-57.0
51178.0
-10.0
51178.0
-345.0
51012.0
-477.0
51066.0
-13.0
51177.0
-118.0
51177.0
-51.0
51177.0
-3.0
51177.0
-6.0
51176.0
-251.0
51058.0
-41.0
51017.0
-271.0
50746.0
-114.0
50632.0
-67.0
50565.0
-53.0
50567.0
-10.0
50567.0
-98.0
50567.0
-175.0
50392.0
-2.0
50392.0
-94.0
50392.0
-56.0
51116.0
-58.0
51307.0
-32.0
51307.0
-191.0
51307.0
-61.0
51246.0
-89.0
51365.0
-72.0
51365.0
-51.0
51365.0
-98.0
51267.0
-119.0
51615.0
-244.0
51371.0
-51.0
51371.0
-410.0
51255.0
-1.0
51204.0
-59.0
51204.0
-38.0
51175.0
-11.0
51230.0
-12.0
51218.0
-2.0
51218.0
-257.0
51341.0
-354.0
51684.0
-150.0
51681.0
-44.0
51681.0
-4.0
51599.0
-26.0
51795.0
-513.0
51816.0
-340.0
51476.0
-1.0
51475.0
-301.0
51174.0
-98.0
51174.0
-80.0
51750.0
-124.0
52340.0
-15.0
52325.0
-77.0
52280.0
-89.0
52212.0
-107.0
52634.0
-12.0


-10.0
49586.0
-244.0
49586.0
-61.0
49586.0
-44.0
49586.0
-190.0
49586.0
-190.0
49586.0
-89.0
49586.0
-51.0
49586.0
-89.0
49586.0
-51.0
49586.0
-10.0
49586.0
-44.0
49586.0
-89.0
49586.0
-190.0
49586.0
-98.0
49586.0
-44.0
49586.0
-89.0
49586.0
-190.0
49586.0
-98.0
49586.0
-244.0
49586.0
-61.0
49586.0
-51.0
49586.0
-44.0
49586.0
-61.0
49586.0
-10.0
49586.0
-38.0
49586.0
-89.0
49586.0
-244.0
49586.0
-61.0
49586.0
-190.0
49586.0
-190.0
49586.0
-61.0
49586.0
-51.0
49586.0
-51.0
49586.0
-244.0
49586.0
-98.0
49586.0
-244.0
49586.0
-10.0
49586.0
-10.0
49586.0
-61.0
49586.0
-61.0
49586.0
-51.0
49586.0
-44.0
49586.0
-10.0
49586.0
-61.0
49586.0
-61.0
49586.0
-190.0
49586.0
-10.0
49586.0
-98.0
49586.0
-61.0
49586.0
-98.0
49586.0
-44.0
49586.0
-38.0
49586.0
-38.0
49586.0
-51.0
49586.0
-98.0
49586.0
-61.0
49586.0
-44.0
49586.0
-44.0
49586.0
-10.0
49586.0
-61.0
49586.0
-244.0
49586.0
-44.0
49586.0
-98.0
49586.0
-89.0
49586.0
-44.0
49586.0
-244.0
49586.0
-44.0
49586.0
-44.0
49586.0
-89.0
49586.0
-10.0


-244.0
49586.0
-10.0
49586.0
-89.0
49586.0
-89.0
49586.0
-61.0
49586.0
-38.0
49586.0
-44.0
49586.0
-89.0
49586.0
-38.0
49586.0
-190.0
49586.0
-98.0
49586.0
-98.0
49586.0
-44.0
49586.0
-61.0
49586.0
-244.0
49586.0
-10.0
49586.0
-89.0
49586.0
-10.0
49586.0
-51.0
49586.0
-89.0
49586.0
-89.0
49586.0
-98.0
49586.0
-10.0
49586.0
-89.0
49586.0
-38.0
49586.0
-89.0
49586.0
-44.0
49586.0
-98.0
49586.0
-51.0
49586.0
-10.0
49586.0
-44.0
49586.0
-10.0
49586.0
-51.0
49586.0
-190.0
49586.0
-244.0
49586.0
-38.0
49586.0
-89.0
49586.0
-190.0
49586.0
-89.0
49586.0
-51.0
49586.0
-38.0
49586.0
-244.0
49586.0
-89.0
49586.0
-61.0
49586.0
-190.0
49586.0
-51.0
49586.0
-89.0
49586.0
-44.0
49586.0
-10.0
49586.0
-10.0
49586.0
-38.0
49586.0
-98.0
49586.0
-44.0
49586.0
-244.0
49586.0
-98.0
49586.0
-51.0
49586.0
-190.0
49586.0
-190.0
49586.0
-98.0
49586.0
-51.0
49586.0
-244.0
49586.0
-44.0
49586.0
-244.0
49586.0
-61.0
49586.0
-89.0
49586.0
-38.0
49586.0
-89.0
49586.0
-190.0
49586.0
-44.0
49586.0
-51.0
49586.0
-61.0


KeyboardInterrupt: 

In [158]:
all_posibble_comb = list(itertools.combinations([i for i in range(8)], 2))
path = [1,2,3,4,5,6,7,8]
cost = calulate_total_cost(path)
for idx1, idx2 in all_posibble_comb:
#             print(idx1, idx2)
#             print('here')
    if (idx2+1)%8 != idx1:
        new_path = path[:idx1] + path[idx1:idx2+1][::-1] + path[idx2+1:] # %100 is const.
        new_cost = dm[path[idx1]][path[(idx2+1)%8]] + dm[path[idx1-1]][path[idx2]] #new edges
        new_cost -= (dm[path[idx1-1]][path[idx1]] + dm[path[idx2]][path[(idx2+1)%8]]) #old edges
        if cost + new_cost != calulate_total_cost(new_path):
            print(idx1, idx2)
            print(path)
            print(new_cost)
            print(new_path)
            break

In [156]:
idx1, idx2 = 0, 7
id
print(path[:idx1], path[idx1:idx2+1][::-1], path[idx2+1:])
new_path = path[:idx1] + path[idx1:idx2+1][::-1] + path[idx2+1:] # %100 is const.
new_cost = dm[path[idx1]][path[(idx2+1)%8]] + dm[path[idx1-1]][path[idx2]] #new edges
new_cost -= (dm[path[idx1-1]][path[idx1]] + dm[path[idx2]][path[(idx2+1)%8]]) #old edges
if cost + new_cost != calulate_total_cost(new_path):
    print(idx1, idx2)
    print(path)
    print(new_cost)
    print(new_path)
      

[] [8, 7, 6, 5, 4, 3, 2, 1] []
0 7
[1, 2, 3, 4, 5, 6, 7, 8]
inf
[8, 7, 6, 5, 4, 3, 2, 1]


In [153]:
calulate_total_cost(new_path)

17162.0